### 深度学习与深层神经网络
- 深度学习：通过多层非线性变换对高复杂度数据建模算法的合集
- 深层神经网络是实现 “多层非线性变换”最常用的一种方法
- 特性：多层 和 非线性
- 线性模型具有局限性，使用非线性模型可以解决线性不可分问题

#### 激活函数实现去线性化
将每个神经元的输出通过一个非线性函数，那么整个神经网络就不再是线性的。神经元输出结果中增加偏置项，然后再使用激活函数。得到输出层的输出

常用激活函数：
- tf.nn.relu
- tf.sigmoind  (logistic)
- tf.tanh

In [1]:
import tensorflow as tf
import numpy as np

c:\users\86132\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\86132\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\86132\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\86132\appd

In [2]:
# 使用激活函数实现反向传播算法
w1 = tf.Variable(tf.random_normal((2, 3), stddev=1, seed=9))
w2 = tf.Variable(tf.random_normal((3, 1), stddev=1, seed=9))
x = tf.placeholder(tf.float32, shape=[None, 2], name='x-input')
biases1 = -0.5
biases2 = 0.1

# 使用配置项和激活函数
# biases1:配置项
a = tf.nn.relu(tf.matmul(x, w1) + biases1)
y = tf.sigmoid(tf.matmul(a, w2) + biases2)
a2 = tf.matmul(x, w1)
y2 = tf.matmul(a2, w2)

# 创建会话计算
with tf.Session() as sess:
    # 初始化
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print('使用激活函数：')
    print(sess.run(y, feed_dict={x: [[0.7, 0.9], [0.1, 0.4], [0.5, 0.8]]}))
    print('未使用激活函数')
    print(sess.run(y2, feed_dict={x: [[0.7, 0.9], [0.1, 0.4], [0.5, 0.8]]}))    

使用激活函数：
[[0.5249792]
 [0.5249792]
 [0.5249792]]
未使用激活函数
[[-0.6053763]
 [-0.6433336]
 [-0.7547992]]


#### 多层网络解决异或运算
1. 感知机可以简单的理解为单层神经网络，感知机会先将输入进行加权和，然后通过激活函数输出
2. 异或运算：两个输入符号相同时，输出0，否则输出1
3. 深层神经网络有组合特征提取的功能，神经网络中加入隐藏层就能很好的解决异或文图

### 损失函数定义
- 神经网络模型的效果及优化的目标时通过损失函数定义的

#### 经典损失函数
1. 神经网络解决多分类问题最常用方法时设置 $n$ 个输出节点，n 为类别个数，对于每个样例神经网络得到一个n维数组作为输出，如果一个样本属于 k 类别，那么此类别对应的输出节点输出值为1，其他节点输出为0。
2. 判断输出向量和期望向量的接近程度：交叉熵 是常用的评判方法之一；给定两个概率分布，$p$和$q$,则交叉熵为：
$$H(p,q)=-\sum_x p(x)\log q(x)$$
3. 当事件总数一定的情况下，概率分布函数$p(X=x)$满足：
$$\forall x  p(X=x)\in [0, 1] 且 \sum_xp(X=x)=1$$
4. Softmax 回归将神经网络向前传播结果变为概率分布；额外的处理层，将神经网络输出变成概率分布。
5. 交叉熵刻画的是两个概率分布之间的距离，值越小两个概率值约接近

In [2]:
# 交叉熵代码实现
#cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
# 其中：
# y_ -- 正确结果
# y  -- 预测结果

# 交叉熵的实现过程中包含4个tensorflow运算：
# 1.tf.clip_by_value(y, 1e-10, 10) -- 将张量控制在一个范围内，避免无效运算 log0。

# 将小于2.4的换成2.4， 大于4.6的换成4.6，使输出结果在给定的区间内
v = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])

with tf.Session() as sess:
    print(sess.run(tf.clip_by_value(v, 2.4, 4.6)))
    
# 2.tf.log 运算：此运算完成了对张量中所有元素依次求对数的功能
    print(sess.run(tf.log(v)))    # 其底为自然数exp
    
# 3.乘法运算：通过 * 相乘 与 矩阵乘法有区别
    v1 = tf.constant([[0.1, 0.2], [0.4, 0.5]])
    v2 = tf.constant([[10.0, 10.0], [10.0, 10.0]])
    print('* 相乘')     
    print(sess.run(v1 * v2))   # 矩阵中对应元素相乘,得到结果
    print('矩阵相乘')
    print(sess.run(tf.matmul(v1, v2)))   # 对应行乘对应列的和
    
# 4.以上步骤得到一个 n×m 的矩阵，n为一个batch中样本的数量，m 为类别的数量，将m个
# 类别的交叉熵相加，再求n个样例得平均交叉熵，即：tf.reduce_mean 函数
    print(sess.run(tf.reduce_mean(v)))   # 对所有元素求平均值
    
# 交叉熵会和softmax一起使用，因此可通过如下函数,：使用了softmax回归之后得交叉熵
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)

[[2.4 2.4 3. ]
 [4.  4.6 4.6]]
[[0.        0.6931472 1.0986123]
 [1.3862944 1.609438  1.7917595]]
* 相乘
[[1. 2.]
 [4. 5.]]
矩阵相乘
[[3. 3.]
 [9. 9.]]
3.5


#### 回归损失函数
1. 交叉熵为解决分类问题稍的回归函数，而回归问题的神经网络只有一个输出节点，这个节点的输出值就是预测值。
2. 对于回归问题，常用的损失函数是*均方误差*（MSE）：
$$MSE(y, y')=\frac{\sum_{i=1}^n(y_i - y'_i)^2}{n}$$

In [7]:
# 回归损失函数代码实现
y_ = tf.constant([[1.0, 3.0, 4.0], [2.0, 1.0, 5.0]])    # 真实值
y = tf.constant([[1.1, 3.2, 4.4], [2.2, 1.1, 5.3]])    # 真实值
mse = tf.reduce_mean(tf.square(y_ - y))   # - 两个矩阵中对应的元素相减

with tf.Session() as sess:
    print(sess.run(mse))

0.05833337


#### 自定义损失函数
1. 目标：使得预测值和真实值间的差距尽量小
2. 使用预测商品销量的例子：
   - 为了最大化利润，需要综合考虑成本和利润，需要将损失函数和利润联系起来，损失函数刻画的应该是成本。
   - 给出如下损失函数：
   $$Loss(y, y')=\sum_{i=1}^n f(y_i, y'_i),   f(x,y)=\begin{cases}a(x-y)  & x>y \\
                                                b(y-x) &x \leq y \end{cases}$$
   - $y_i$为实际结果，$y_i '$为预测结果，a 和 b 是常量，
   - 通过代码实现：loss = tf.reduce_sum(tf.where(tf.greater(v1, v2), a * (v1 - v2), b * (v2 - v1))

In [8]:
# tf.where 和tf.greater 用法
v1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
v2 = tf.constant([4, 3, 2, 1], dtype=tf.float32)

with tf.Session() as sess:
    print(sess.run(tf.greater(v1, v2)))   # 比较两个张量中的大小，并返回结果
    print(sess.run(
        tf.where(tf.greater(v1, v2), v1, v2)  
    ))
    # 三个参数，第一个参数为真，返回第二值；否则返回第三个值

W1017 12:55:40.051807  7264 deprecation.py:323] From <ipython-input-8-2cf95fa6f465>:8: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[False False  True  True]
[4. 3. 3. 4.]


In [19]:
# 实现一个简单的神经网络
# 输入 x: 样本特征
# 输出 y: 预测商品数量
# 求解：求解最优参数，使得损失函数尽可能小
batch_size = 8

# 输入两个节点，样本数据和真实结果
x = tf.placeholder(tf.float32, shape=(None, 2), name='x_input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y_input')

# 定义一个单层的神经网络向前传播过程
w1 = tf.Variable(tf.random_normal((2, 1), stddev=1.0, seed=9))
y = tf.matmul(x, w1)

# 预测成本，少预测一件的成本是10，多预测一件的成本是1
loss_less = 10 
loss_more = 1
# 损失函数
loss = tf.reduce_sum(tf.where(tf.greater(y, y_), 
                             (y - y_) * loss_more,
                             (y_ - y) * loss_less))
# 反向传播优化函数
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

# 模拟数据
rdm = np.random.RandomState(9)
dataset_size = 128
X = rdm.rand(dataset_size, 2)

# 加入噪音数据
Y = [[x1 + x2 + rdm.rand()/10.0 - 0.05] for x1, x2 in X]

# 训练神经网络
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run(w1))
    for i in range(5000):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        sess.run(train_step,
                 feed_dict={x: X[start:end], y_: Y[start:end]})
    w2 = sess.run(w1)

[[ 1.1550728]
 [-0.4910573]]


In [22]:
# 使用最优参数预测数据
w2 = tf.constant(w2, dtype=tf.float32)
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run(w2))
    x_test = tf.constant(X[:5], dtype=tf.float32)
    print(sess.run(tf.matmul(x_test, w2)))

[[1.0411468]
 [1.0336019]]
[[0.5295395 ]
 [0.65449923]
 [0.37386113]
 [0.69216627]
 [0.44462648]]


### 神经网络优化算法
- 梯度下降法（gradient decent）：主要用于优化单个参数取值
- 反向传播算法（backpropagation）：在所有的参数上使用梯度下降法，神经网络中的核心算法

#### BP算法
- 梯度下降法 参数更新公式：
$$\theta_{n+1} = \theta_n - \eta \frac {\Delta}{\Delta\theta_n} J(\theta_n)$$
- 神经网络训练步骤：
   1. 读取小部分数据作为当前训练数据执行反向传播算法
   2. 定义神经网络结果和优化算法
   3. 训练神经网络，迭代更新参数

### 神经网络进一步优化


#### 学习率设置
- 指数衰减法：先使用较大的学习率，得到一个较优解，然后随着迭代减小学习率
- 指数衰减发使用样例，如下：

In [ ]:
# 初始学习率为0.1，训练100轮后，学习率乘以0.96

global_step = tf.Variable(0)

# 使用函数生成学习率
learning_rate = tf.train.exponential_decay(
    0.1, global_step, 100, 0.96, staircase=True)
# 使用指数衰减的学习率，将其传入函数中，将自动更新
learning_step = tf.train.GradientDescentOptimizer(learning_rate)\
                .minimize(my_loss, global_step=global_step)

#### 过拟合问题
- 正则化思想：在损失函数中加入刻画模型复杂度的指标，加入损失函数为$J(\theta)$，那么不直接优化$J(\theta)$，而是优化$J(\theta)+\lambda R(\omega)$。
 $R(\omega)$刻画的是模型的复杂程度，$\lambda$是模型复杂损失在总损失上中的占比

- 常见的优化函数两种：
    1. L1正则化：会让参数变得稀疏，不可导
    2. L2正则化：正则化计算公示可导
- L1 和 L2正则可以同时使用:$R(\omega) = \sum_i \alpha|\omega_i| + (1-\alpha)\ \omega_i^2$

In [4]:
# tensorflow 中使用正则化函数
weights = tf.constant([[1.0, -2.0], [-3.0, 4.0]])
with tf.Session() as sess:
    # 权重的绝对值和(|1| + |-2| + |-3| + |4|) * 0.5 = 5
    print(sess.run(tf.contrib.layers.l1_regularizer(.5)(weights)))
    # 权重的平方和 （(1^2 + (-2)^2 + (-3)^2 + 4^2)/2） * 0.5 = 
    print(sess.run(tf.contrib.layers.l2_regularizer(.5)(weights)))

5.0
7.5


In [ ]:
# 
def get_weight(shape, lambda):
    # 生成一个变量
    var = tf.